# 模型量化处理
* 数据选择

![](https://s2.loli.net/2026/01/21/aXEqLHontgUS1Pd.png)

In [1]:
import os
import subprocess
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## 使用llama.cpp量化
### 基本原理解释
#### 1、量化精度

#### 2、量化范式
### 1、量化大语言模型
> https://qwen.readthedocs.io/zh-cn/latest/quantization/llama.cpp.html

为了实现GGUF格式转换，按照下面步骤进行
#### 1、无校准量化GGUF
```python
# 首先创建GGUF文件  转换文件
python convert-hf-to-gguf.py 模型名称/地址 --outtype bf16 --outfile Qwen3-1.5B-BF16.gguf 
# 其中的 outtype后面的数据类型可以是：bf16 f32等
```
在执行上面代码之后直接将模型量化到`8 bit`：
```python
./build/bin/llama-quantize --quantize --model Qwen3-1.5B-F16.gguf --output Qwen3-1.5B-Q8_0.gguf --type Q8_0
```
`Q8_0`是一个量化预设的代号
#### 2、使用重要性矩阵量化GGUF
```python
# 首先
./llama-imatrix -m Qwen3-1.5B-F16.gguf -f calibration-text.txt --chunk 512 -o Qwen3-1.5B-imatrix.dat -ngl 80
# 
./llama-quantize --imatrix Qwen3-1.5B-imatrix.dat Qwen3-1.5B-F16.gguf Qwen3-1.5B-Q4_K_M.gguf Q4_K_M
```
#### 两种量化方式代码解释
在此之前可以使用`python convert_hf_to_gguf.py --help`了解一些这个脚本都有哪些参数，`python convert_hf_to_gguf.py --print-supported-models`直接看一下支持哪些模型进行量化，一般而言模型名称就和`config.json`中是一致的比如`Qwen2ForCausalLM`。量化开始：**首先**，在下载得到模型权重之后，在`cache_dir`里面就会有模型权重，比如说上面过程中模型权重就会保存为：`/Model/models--Qwen--Qwen2.5-1.5B-Instruct`，那么在创建以及转换文件过程中就需要将其中的 **模型名称/地址**进行替换，那么**完整带脚本**就是：`python convert_hf_to_gguf.py /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct --outfile /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf --outtype bf16 --verbose`
**不过**，上面代码还是会出错，因为实际模型文件在`code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306`因此需要将上面的文件路径改为：`/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306`最后输出：
```bash
...
INFO:hf-to-gguf:Model successfully exported to /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf
```
就表示成功了，而后可以通过`./build/bin/llama-quantize --help`去检查支持哪些参数，**并且进行量化**：`./build/bin/llama-quantize /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf q8_0`。不过还有一点有意思的，比如说我的模型一般都会用lora进行微调，我需要对lora微调模型进行处理，类似的：
```bash
python convert_lora_to_gguf.py /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/test-lora/ --outfile /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf --outtype bf16 --verbose

./build/bin/llama-quantize /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-Q8_0.gguf q8_0
```
> **总结上面过程代码**：
> `python convert_hf_to_gguf.py /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306 --outfile /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf --outtype bf16 --verbose`
> `./build/bin/llama-quantize /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf q8_0`

### 使用GGUF量化模型
```bash
pip install llama-cpp-python
# 直接去 https://github.com/abetlen/llama-cpp-python/releases 下载对应的版本 和flash-attn一样
pip install llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl
```
这样一来就安装好了就可以直接加载GGUF模型权重了（和非量化模型推理10.8s，**量化后推理：3.5s**）：
```python
from llama_cpp import Llama

llm = Llama(
    model_path="/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf",
    # lora_path= 
    n_ctx=8192,
    n_gpu_layers=-1,
    chat_format="chatml",
    verbose=False
)

messages = [
    {"role": "system", "content": "你是一个有帮助的助手，用简洁的中文回答。"},
    {"role": "user",   "content": "晚上睡不着怎么办"}
]

# 生成
response = llm.create_chat_completion(
    messages,
    max_tokens=300,
    temperature=0.75,
    top_p=0.95,
    repeat_penalty=1.1,
    stream=False  # 改成 True 即可流式输出
)

print("AI 回答：")
print(response["choices"][0]["message"]["content"])
```

#### 2、量化范式

In [8]:
import os
import subprocess
from transformers import AutoTokenizer, AutoModelForCausalLM
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

def load_model():
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    return model, tokenizer

def load_GGUF(lora_path=None):
    from llama_cpp import Llama

    llm = Llama(
        model_path="/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf",
        lora_path= lora_path,
        n_ctx=8192,
        n_gpu_layers=-1,
        chat_format="chatml",
        verbose=False
    )
    return llm
messages = [
    {"role": "system", "content": "你是一个有帮助的助手，用简洁的中文回答。"},
    {"role": "user",   "content": "晚上睡不着怎么办?"}
]
model, tokenizer = load_model()
model_gguf = load_GGUF()
model_gguf_lora = load_GGUF(lora_path= '/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf')

llama_context: n_ctx_per_seq (8192) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (8192) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


## 使用llmcompressor进行模型量化
> `llmcompressor==0.7.1、transformers==4.55.2、torch==2.8.0+cu126、trl==0.22.2、vllm==0.11.0` 
### 原理及官方文档总结

| 方法 | 描述 | 准确性恢复 vs. 时间 |
|---|---|---|
| GPTQ | 利用二阶逐层优化来优先处理重要的权重/激活，并允许更新剩余权重 | 准确性恢复高，但运行成本更高/更慢 |
| AWQ | 使用通道级缩放更好地保留权重和激活中的重要异常值 | 比GPTQ具有更好的准确性恢复和更快的运行时间 |
| SmoothQuant | 通过将异常值折叠到权重中来平滑激活中的异常值，确保权重和激活量化模型具有更好的准确性 | 准确性恢复良好，校准时间最短；可与其他方法组合使用 |
| 就近取整(RTN) | 简单的量化技术，将每个值四舍五入到目标精度中最接近的可表示级别 | 在大多数情况下提供中等准确性恢复。计算成本低且实现速度快，适用于实时或资源受限的环境 |
| AutoRound | 通过符号梯度下降优化舍入和裁剪范围 | 与GPTQ/AWQ相比，提供领先的4位和优异的低于4位精度，运行时间比GPTQ快，与AWQ相当 |

直接对[官方](https://docs.vllm.ai/projects/llm-compressor/en/0.7.1/getting-started/compress/#apply-the-recipe)中描述进行总结，收其提供的压缩方式进行操作，直接使用llmcompressor中的压缩方式使用起来很简单，可以直接使用`oneshot`进行量化，比如说：
```python
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot

recipe = [
    SmoothQuantModifier(smoothing_strength=0.8),
    GPTQModifier(scheme="W8A8", targets="Linear", ignore=["lm_head"]),
]
oneshot(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset="open_platypus",
    recipe=recipe,
    output_dir="TinyLlama-1.1B-Chat-v1.0-INT8",
    max_seq_length=2048,
    num_calibration_samples=512,
)
```
**值得注意的是**，在使用其进行模型压缩过程中对于显存的估计大概是：`mem(1B parameters) ~= (1B parameters) * (2 bytes / parameter) = 2B bytes ~= 2Gb`，除此之外压缩模型过程中对于文本解码层（LLM部分）以及视觉编码层（Vision Layer部分）两部分压缩过程中处理存在差异，**前者**会动态地将模型逐层加载到GPU进行计算，其余部分仍保留在CPU内存中。**后者**会一次性全部加载到GPU。与文本模型不同，视觉编码层在加载到GPU前不会被拆分为独立层。对于视觉塔层规模大于文本层的模型，这种机制可能导致GPU内存瓶颈。目前（**测试的是0.7.1版本实际对于最新的也是如此**） LLM Compressor 暂不对视觉编码器进行量化处理，因量化技术在延迟/吞吐量与精度损失间的权衡通常得不偿失。比如说：
**1、四舍五入量化（Round To Nearest RTN）**  
![](https://s2.loli.net/2026/01/20/JiXhKPoFfpNdv7e.png)  
**2、GPTQ**  
![](https://s2.loli.net/2026/01/20/m2IqEYWsFKUPbkT.png)  
在llm compressor中支持量化操作  
![](https://s2.loli.net/2026/01/20/br2FjTWAOXp3nU5.png)    
对于[上述内容解释](https://github.com/vllm-project/llm-compressor/blob/main/docs/guides/compression_schemes.md?plain=1)：  

| Scheme | Description | Data required? | vLLM Hardware Compatibility |
|--------|-------------|----------------|-----------------------------|
| **[W8A8-FP8](../../examples/quantization_w8a8_fp8/README.md)** | 8位浮点（FP8）量化用于权重和激活值，通过8位算术运算实现约2倍的权重压缩。采用逐通道（channel-wise）量化将权重压缩至8位，并采用动态逐token或静态逐张量（per-tensor）量化将激活值压缩至8位。权重的缩放因子既可以按通道生成，也可以按张量生成。其中，逐通道权重量化配合动态逐token激活量化是性能最优的选项。W8A8-FP8无需校准数据集，激活值的量化在vLLM推理过程中实时进行。该方案在通用性能和压缩率方面表现良好，尤其适用于服务器端和批处理推理场景。 | 无需校准数据集，除非你采用静态逐张量激活量化。 | 支持最新的 NVIDIA GPU（Hopper 架构及之后）和最新的 AMD GPU。推荐用于计算能力 >=8.9 的 NVIDIA GPU（包括 Hopper、Ada Lovelace 和 Blackwell 架构）。 |
| **[W8A8-INT8](../../examples/quantization_w8a8_int8/README.md)** | 8位整数（INT8）量化用于权重和激活值，通过8位算术运算实现约2倍的权重压缩。使用GPTQ进行逐通道（channel-wise）量化将权重压缩至8位，并采用动态逐token量化将激活值压缩至8位。INT8权重量化既支持逐张量（per-tensor）也支持逐通道（per-channel）方式。W8A8-INT8在通用性能和压缩率方面表现良好，尤其适用于服务器端和批处理推理。激活值的量化在vLLM推理过程中执行，可选择静态或动态方式；此外，INT8激活值还支持非对称量化。W8A8-INT8有助于在高QPS场景或使用vLLM进行离线服务时提升推理速度。 | 权重量化以及静态逐张量激活量化需要校准数据集 | 支持所有NVIDIA GPU、AMD GPU、TPU、CPU及其他加速器。推荐用于计算能力 <8.9 的NVIDIA GPU（如Ampere、Turing、Volta、Pascal或更早架构）。 |
| **[W4A16](../../examples/quantization_w4a16/README.md)** | 仅将权重量化为4位整数（INT4）精度，激活值保持16位浮点（FP16）精度。W4A16可实现约3.7倍的权重压缩，但仍需使用16位算术运算。W4A16还支持非对称权重量化。该方案在内存受限、对延迟敏感的应用中提供最大压缩率，并在低QPS场景下通过更高的权重压缩带来显著加速。所附示例采用GPTQ算法以降低量化损失，但也可使用其他算法，如[AWQ](../../examples/awq/llama_example.py) 进行| 需要数据集 | 所有设备 |
| **W8A16** | 将模型权重编码为8位整数，激活值编码为16位整数。W8A16压缩相比FP32可生成更小的模型输出体积，并在具备原生8位整数计算单元的硬件上实现更快的推理速度。相比浮点运算，其功耗和内存带宽需求更低| 需要数据集 | 所有设备 |
| **NVFP4** | NVFP4 是随 NVIDIA Blackwell GPU 架构推出的4位浮点编码格式。它通过高精度缩放编码和两级微块（micro-block）缩放策略，在张量值的宽动态范围内保持数值精度。NVFP4 压缩为每个张量生成一个全局缩放因子，并为每16个元素组成的组生成局部量化缩放因子。权重和激活值均使用全局缩放因子和局部量化缩放因子进行量化。组大小固定，不可更改。 | 需要数据集 | 仅支持 NVIDIA Blackwell 架构及后续 GPU |
| [**W8A8-FP8_BLOCK**](../../examples/quantization_w8a8_fp8/fp8_block_example.py)| 采用分块量化（通常为 128×128 的块）将权重压缩至 FP8，并对激活值使用动态逐 token 组（每组 128 个 token）量化。无需校准数据集，激活量化在 vLLM 推理过程中实时执行 | 不需要数据集 | 支持最新的 NVIDIA GPU（Hopper 及之后）和最新的 AMD GPU。推荐用于计算能力 ≥8.9 的 NVIDIA GPU（包括 Hopper、Ada Lovelace 和 Blackwell 架构） |
| **[2:4 Semi-structured Sparsity](../../examples/sparse_2of4_quantization_fp8/README.md)** | U采用半结构化稀疏（如 SparseGPT）：在张量中每连续 4 个权重中，将其中 2 个置零。使用权重逐通道量化压缩至 8 位，并对激活值采用动态逐 token 量化压缩至 8 位。相比标准 W8A8-FP8，可实现更优推理性能，且评估分数几乎无下降，参考[evaluation score](https://neuralmagic.com/blog/24-sparse-llama-fp8-sota-performance-for-nvidia-hopper-gpus/). 但小型模型可能因剩余非零权重不足以还原原始分布而出现精度下降。| 需要数据集 |  支持所有 NVIDIA GPU、AMD GPU、TPU、CPU 及其他加速器。推荐用于计算能力 ≥9.0 的 GPU（Hopper 和 Blackwell 架构） |
| **Unstructured Sparsity** | 非结构化稀疏量化以无固定模式的方式将模型中的个别权重置零。与分块或通道剪枝不同，它在模型中任意位置移除贡献最小的权重，从而生成细粒度的稀疏矩阵。 |不需要数据集 | 支持所有 NVIDIA GPU、AMD GPU、TPU、CPU 及其他加速器 |

### 关键参数解释
介绍其支持量化范式之后直接看代码操作
```python
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot

recipe = [
    SmoothQuantModifier(smoothing_strength=0.8),
    GPTQModifier(scheme="W8A8", targets="Linear", ignore=["lm_head"]),
]
oneshot(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset="open_platypus",
    recipe=recipe,
    output_dir="TinyLlama-1.1B-Chat-v1.0-INT8",
    max_seq_length=2048,
    num_calibration_samples=512,
)
```
#### 1、oneshot参数解释
**首先**看`oneshot`中参数操作，在代码`llmcompressor/entrypoints/oneshot.py`以及[文档](https://docs.vllm.ai/projects/llm-compressor/en/0.7.1/reference/llmcompressor/entrypoints/oneshot/?h=oneshot)中参数，着重了解：  
* `model`：直接使用hf上模型名称/本地名称，对于本地需要具体到config.json位置比如说：`models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa30/`  
* `cache_dir`：缓存路径：str
* `output_dir`：保存路径：str
* `log_dir`: 日志路径：str
* `text_column`：指定数据的columns（也就是用数据集中那一个标签的数据量化处理）
* `dataset`：使用的数据集，str, Dataset, DatasetDict，如果在直接使用hf上数据集那么直接用名称就行，如open_platypus。（具体代码见下）对于后面两项可以直接用 `datasets`中load方式加载数据即可

此处简单介绍其中**数据处理逻辑**，在`llmcompressor`中数据的处理逻辑和`datasets`库（该库一般就是两个核心操作load、process，前者加载数据后者直接编码数据）处理逻辑一致，datasets中**加载本地数据**[具体逻辑](https://huggingface.co/docs/datasets/loading)如下：**1、如果就只有一个文件**（比如说我的所有数据都存储在一个jsonl中），那么可以直接使用`load_dataset('json', data_files= './test_datasets.jsonl',streaming=True)`;**2、如果有多个文件**，比较简单方式直接将`data_files`替换为`tmp/TestDatasets-Json/*.jsonl`其中用 * 通配符去匹配所有的jsonl文件，除此之外还可以**用一个字典去表示**，比如说：`{'train': [xxx, ..., xxx]}`
> 上面代码中用 steaming=True 流式处理去处理数据较大情况，访问数据可以直接`next(iter(json_one_dataset["train"]))`

> **处理数据**，将数据丢给llm/vllm进行处理之前，需要将数据进行变换，比如说转化为对应的数据格式/对数据进行tokenizer进行处理，这里处理方式在`datasets`中可以直接使用map处理即可（具体处理看下面例子）。回到`oneshot`中的数据处理过程中，其内部逻辑和`datasets`中处理逻辑一致，处理本地文件逻辑如下：`get_processed_dataset`(llmcompressor/datasets/utils.py)判断数据是不是提前被tokenized处理，**如果是**那么直接返回即可，**如果不是**直接使用`TextGenerationDataset.load_from_registry`(llmcompressor/transformers/finetune/data/base.py)处理数据  
==>`TextGenerationDataset`处理数据逻辑，加载数据-->process数据（两部分数据和hf库datasets的处理逻辑是一致的），对于第一部分**加载数据处理如下**：
1、对于hf上数据直接加载==>返回  
2、对于自定义的数据（也支持使用下载好的hf本地数据），使用`get_custom_datasets_from_path`(llmcompressor/transformers/finetune/data/data_helpers.py)，在这个函数中默认两个参数：path、ext（文件类型）。对于path格式要求  
![](https://s2.loli.net/2026/01/20/my5v1aBonNuh9IZ.png)  
最后都统一返回。**process数据处理过程**和`datasets`中处理是一致的，再代码中`TextGenerationDataset`(llmcompressor/transformers/finetune/data/base.py)中直接使用`self.map()`处理数据而后再去通过`format_calibration_data`（llmcompressor/datasets/utils.py）使用DataLoader进行处理
#### 2、量化方式参数解释

### AWQ量化
[https://www.big-yellow-j.top/posts/2025/10/11/Quantized.html](https://www.big-yellow-j.top/posts/2025/10/11/Quantized.html)
### GPTQ量化
[https://www.big-yellow-j.top/posts/2025/10/11/Quantized.html](https://www.big-yellow-j.top/posts/2025/10/11/Quantized.html)
### 1、量化大语言模型

### 2、量化多模态模型
### 量化模型使用


In [10]:
import os
from datasets import load_dataset

# hf_dataset = load_dataset("garage-bAInd/Open-Platypus", 
#                        split="train",
#                        cache_dir= '/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/')
# print(f"{hf_dataset[0]}")

path_dir = './tmp/TestDatasets-Json/'
json_dir_dataset = load_dataset('json', 
                                 data_files= {'train': [os.path.join(path_dir, f) for f in os.listdir(path_dir)]},)

json_one_dataset = load_dataset('json', 
                                 data_files= './test_datasets.jsonl',
                                 streaming=True)
json_dict_dataset = load_dataset('json', 
                                 data_files= {'train': [os.path.join(path_dir, f) for f in os.listdir(path_dir)],
                                              'test': [os.path.join(path_dir, f) for f in os.listdir(path_dir)]})

# print(json_one_dataset, '\n', json_dir_dataset, '\n', json_dict_dataset)
print(next(iter(json_one_dataset["train"])), '\n', json_dict_dataset['train'][0])

# 处理数据
def mulitmodel_data_process(data):
    '''处理多模态数据'''
    messages = [
        {"role": "system","content": [{"type": "text", "text": "You are a helpful multimodal assistant."}]},
        {"role": "user","content": [
            {"type": "image", "image": data['image']},
            {"type": "text", "text": data['prefix']},],},
        {"role": "assistant","content": [
            {"type": "text", "text": data['suffix']}],},]
    return {"messages": messages}


json_dict_dataset_map = json_dict_dataset.map(mulitmodel_data_process)
json_dict_dataset_map['train'][0]

{'image': '../data/OmniDoc/images/eastmoney_1885ca41425d245551f3482457304f78b48186bff625fa91e675eaf6bba5229f.pdf_0.jpg', 'height': 2338, 'width': 1654, 'prefix': 'QwenVL HTML', 'suffix': '<body><h2 data-bbox="67 227 1138 289">索通发展 Sunstone Development（603612 CH）</h2><h2 data-bbox="65 297 1566 355">首次覆盖：扩张阳极产能稳固龙头地位，布局负极培育第二成长曲线</h2><h2 data-bbox="64 364 1597 449">Expand Anode Production Capacity to Stabilize the Leading Position, and Lay out the Second Growth Curve of Anode Cultivation: Initiation</h2><p data-bbox="197 484 517 516">观点聚焦 Investment Focus</p><h2 data-bbox="90 536 691 570">首次覆盖优于大市 Initiate with OUTPERFORM</h2><div class="table" data-bbox="69 571 703 885"><table><tr><td>评级</td><td>优于大市OUTPERFORM</td></tr><tr><td>现价</td><td>Rmb18.38</td></tr><tr><td>目标价</td><td>Rmb24.50</td></tr><tr><td>市值</td><td>Rmb9.94bn/US$1.36bn</td></tr><tr><td>日交易额（3个月均值）</td><td>US$15.46mn</td></tr><tr><td>发行股票数目</td><td>540.85mn</td></tr><tr><td>自由流通股（%）</td><td>50%</td></tr><tr><td>1年股价最高最低值</td>

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot

recipe = [
    SmoothQuantModifier(smoothing_strength=0.8),
    GPTQModifier(scheme="W8A8", targets="Linear", ignore=["lm_head"]),
]
# 直接使用hf上数据
# oneshot(
#     model='/root/autodl-tmp/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306',#"Qwen/Qwen2.5-1.5B-Instruct",
#     cache_dir= '/root/autodl-tmp/Model',
#     dataset="open_platypus",
#     recipe=recipe,
#     output_dir="./tmp/LLMCompressor-GPTQ-W8A8-0.8",
#     log_dir= './tmp/LLMCompressor-GPTQ-W8A8-0.8/log',
    
#     max_seq_length=2048,
#     num_calibration_samples=512,
# )
# 直接使用自定义数据集 其支持参数类型为 Optional[Union[str, "Dataset", "DatasetDict"]] 那么就不能使用 流处理加载数据

def to_prompt(example):
    return {"prompt": f"{example['prefix']} {example['suffix']}"}
json_dict_dataset = json_dict_dataset.map(to_prompt)
oneshot(
    model='/root/autodl-tmp/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306',#"Qwen/Qwen2.5-1.5B-Instruct",
    cache_dir= '/root/autodl-tmp/Model',
    dataset= json_dict_dataset,
    text_column="prompt",

    recipe=recipe,
    output_dir="./tmp/LLMCompressor-GPTQ-W8A8-0.8",
    log_dir= './tmp/LLMCompressor-GPTQ-W8A8-0.8/log',
    
    max_seq_length=2048,
    num_calibration_samples=512,
)

Tokenizing: 100%|██████████| 3/3 [00:00<00:00, 123.23 examples/s]

2026-01-21T23:29:40.431532+0800 | format_calibration_data | WARNING - Requested 512 calibration samples but the provided dataset only has 3. 
2026-01-21T23:29:40.438831+0800 | reset | INFO - Compression lifecycle reset
2026-01-21T23:29:40.443245+0800 | from_modifiers | INFO - Creating recipe from modifiers
2026-01-21T23:29:40.444274+0800 | _infer_mappings_from_model | INFO - No SmoothQuantModifier.mappings provided, inferring from model...


2026-01-21T23:29:41.771372+0800 | initialize | INFO - Compression lifecycle initialized for 2 modifiers
2026-01-21T23:29:41.773017+0800 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `SmoothQuantModifier`


(1/29): Calibrating: 100%|██████████| 3/3 [00:00<00:00,  3.79it/s]

2026-01-21T23:29:43.289540+0800 | _apply_smoothing | INFO - Smoothing with model.layers.0.input_layernorm
2026-01-21T23:29:43.304885+0800 | _apply_smoothing | INFO - Smoothing with model.layers.0.post_attention_layernorm



(1/29): Propagating: 100%|██████████| 3/3 [00:00<00:00, 16.30it/s]


KeyboardInterrupt: 

In [ ]:
import multiprocessing
from vllm import LLM, SamplingParams

multiprocessing.set_start_method('spawn', force=True)
model = LLM("/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/Qwen2.5-1.5B-GPTQ-W8A8")

# output = model.generate("晚上睡不着怎么办?")
# output[0].outputs[0].text
sampling_params = SamplingParams(
    temperature=0.8,           # 温度：值越高输出越随机
    top_p=0.9,                # 核采样：控制词汇多样性
    top_k=50,                 # Top-k采样
    max_tokens=1024,          # 最大生成长度
    frequency_penalty=0.1,    # 频率惩罚：减少重复内容
    presence_penalty=0.1,     # 存在惩罚：鼓励新话题
    repetition_penalty=1.1,   # 重复惩罚（某些模型支持）
)

output = model.generate("晚上睡不着怎么办?", sampling_params)
print(output[0].outputs[0].text)

/root/autodl-tmp/miniconda3/envs/docparse/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 01-22 22:35:44 [__init__.py:216] Automatically detected platform cuda.
INFO 01-22 22:35:48 [utils.py:233] non-default args: {'disable_log_stats': True, 'model': '/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/Qwen2.5-1.5B-GPTQ-W8A8'}
INFO 01-22 22:35:48 [model.py:547] Resolved architecture: Qwen2ForCausalLM
INFO 01-22 22:35:48 [model.py:1510] Using max model len 32768


2026-01-22 22:35:54,970	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 01-22 22:35:54 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=677787) INFO 01-22 22:35:56 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=677787) INFO 01-22 22:35:56 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/Qwen2.5-1.5B-GPTQ-W8A8', speculative_config=None, tokenizer='/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/Qwen2.5-1.5B-GPTQ-W8A8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.20it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.20it/s]
(EngineCore_DP0 pid=677787) 


(EngineCore_DP0 pid=677787) INFO 01-22 22:36:00 [default_loader.py:267] Loading weights took 0.97 seconds
(EngineCore_DP0 pid=677787) INFO 01-22 22:36:01 [gpu_model_runner.py:2653] Model loading took 2.1034 GiB and 1.283012 seconds
(EngineCore_DP0 pid=677787) INFO 01-22 22:36:12 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/c3c5955daf/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=677787) INFO 01-22 22:36:12 [backends.py:559] Dynamo bytecode transform time: 9.80 s
(EngineCore_DP0 pid=677787) INFO 01-22 22:36:17 [backends.py:197] Cache the graph for dynamic shape for later use
(EngineCore_DP0 pid=677787) INFO 01-22 22:36:53 [backends.py:218] Compiling a graph for dynamic shape takes 40.38 s
(EngineCore_DP0 pid=677787) INFO 01-22 22:37:01 [monitor.py:34] torch.compile takes 50.18 s in total
(EngineCore_DP0 pid=677787) INFO 01-22 22:37:03 [gpu_worker.py:280] The init_snapshot total_memory: 23.55 GiB The non_kv_cache_memory: 3.78 GiB
(Engine

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:03<00:00, 21.53it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 17.58it/s]


(EngineCore_DP0 pid=677787) INFO 01-22 22:37:11 [gpu_model_runner.py:3480] Graph capturing finished in 7 secs, took 0.65 GiB
(EngineCore_DP0 pid=677787) INFO 01-22 22:37:11 [core.py:210] init engine (profile, create kv cache, warmup model) took 70.23 seconds
INFO 01-22 22:37:14 [llm.py:306] Supported_tasks: ['generate']


In [ ]:
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

llm_model_path = "/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/Qwen2.5-1.5B-GPTQ-W8A8"
model = AutoModelForCausalLM.from_pretrained(llm_model_path,device_map="auto",)
tokenizer = AutoTokenizer.from_pretrained(llm_model_path)

gen_config = GenerationConfig(
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.05,
    do_sample=True,
)

SAMPLE_PROMPTS = [
    "晚上睡不着怎么办？",
    "写一段关于秋天的诗",
    "解释一下量子纠缠是什么",
]

for prompt in SAMPLE_PROMPTS:
    s_time = time.time()
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": prompt},
    ]
    
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    output_ids = model.generate(**inputs, generation_config=gen_config)
    
    response = tokenizer.decode(output_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    print(f"\nPrompt: {prompt}")
    print(f"回复({time.time()- s_time}): {response.strip()}\n{'-'*60}")

/root/autodl-tmp/miniconda3/envs/docparse/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`generation_config` default values have been modified to match model-specific defaults: {'top_k': 20, 'pad_token_id': 151643, 'bos_token_id': 151643, 'eos_token_id': [151645, 151643]}. If this is not desired, please set these values explicitly.



Prompt: 晚上睡不着怎么办？
回复(45.25086069107056): 这里有一些建议，可能对你有帮助：

1. **尝试放松技巧**：深呼吸、渐进性肌肉放松或冥想等方法可以帮助你减轻焦虑和压力。

2. **限制咖啡因和酒精的摄入**：特别是在睡前，避免饮用含咖啡因的饮料（如咖啡、茶）和酒精，因为它们可能会干扰你的睡眠。

3. **保持良好的睡眠卫生**：确保你的睡眠环境舒适、安静且黑暗。使用舒适的床上用品，并尽量在相同的时间上床睡觉和起床。

4. **定期锻炼**：尽管睡前锻炼可能会使你更加清醒，但定期的轻度至中度运动可以帮助你改善睡眠质量。

5. **晚餐选择**：避免晚餐过晚或过重，特别是避免摄入辛辣食物，因为它们可能会引起胃部不适，从而影响睡眠。

6. **记录日志**：有时写下你的想法和担忧，可以帮助你更好地理解自己的思维模式，从而更容易
------------------------------------------------------------

Prompt: 写一段关于秋天的诗
回复(33.00411057472229): 秋风起，落叶舞，
稻田金黄，丰收喜。
山河换新装，色彩斑斓，
秋天来了，带来希望。

果实累累挂枝头，
菊花绽放，香气扑鼻。
寒露滴滴，滴落心间，
秋天的美，让人陶醉。

天空高远，白云悠闲，
秋水共长天一色。
人们穿着轻盈，踏着秋韵，
享受着这个季节的静谧与温柔。

夜晚降临，月色皎洁，
星星闪烁，宛如繁星点点。
秋夜宁静，万物沉睡，
让心灵得以净化，回归自然。

秋天，你是大自然的杰作，
你是生命的赞歌，是希望的使者。
在这个收获的季节里，
让我们一同品味，这份美好的馈赠。
------------------------------------------------------------

Prompt: 解释一下量子纠缠是什么
回复(16.59041476249695): 量子纠缠是一种量子力学现象，指两个或多个粒子之间形成一种特殊的关系，使得它们的某些性质在空间上是相关联的。当其中一个粒子的状态改变时，另一个粒子的状态也会立即发生变化，即使它们相隔很远的距离。

量子纠缠是一个非常有趣和神秘的现象，因为它超越了经典物理学的理解，并且在量子计算、量子通信等领域有广泛的应用。
-----

: 

## 模型微调
> https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb

https://github.com/shangxiaaabb/ProjectCode/blob/main/code/Python/DFModelCode/DF_acceralate/GRPO_training.py

## 模型加载

In [ ]:
import os
import subprocess
from transformers import AutoTokenizer, AutoModelForCausalLM
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

def load_model():
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    return model, tokenizer

def load_GGUF(lora_path=None):
    from llama_cpp import Llama

    llm = Llama(
        model_path="/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf",
        lora_path= lora_path,
        n_ctx=8192,
        n_gpu_layers=-1,
        chat_format="chatml",
        verbose=False
    )
    return llm
messages = [
    {"role": "system", "content": "你是一个有帮助的助手，用简洁的中文回答。"},
    {"role": "user",   "content": "晚上睡不着怎么办?"}
]
model, tokenizer = load_model()
model_gguf = load_GGUF()
model_gguf_lora = load_GGUF(lora_path= '/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf')

In [9]:
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

可以试试深呼吸、听轻音乐、喝杯热牛奶或泡个热水澡等方法。如果经常失眠，建议咨询医生。<|im_end|>


In [10]:
response = model_gguf.create_chat_completion(
    messages,
    max_tokens=300,
    temperature=0.75,
    top_p=0.95,
    repeat_penalty=1.1,
    stream=False  # 改成 True 即可流式输出
)

print("AI 回答：")
print(response["choices"][0]["message"]["content"])

AI 回答：
可尝试用热水泡脚，听听轻音乐，或者看看电视。
